In [44]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""

import requests
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

import time

import re
from datetime import datetime

In [45]:
#read csv

data = pd.read_csv("markets_zusatz.csv", sep = ";")

df = pd.DataFrame(data)

df.head()

,id,name,shortname,strasse,plz_ort,bezirk,veranstalter,von,bis,oeffnungszeiten,email,w3,bemerkungen,lat,lng,rss_titel,rss_beschreibung,barrierefrei,immer-kostenlos,Mo,Di,Mi,Do,Fr,Sa,So,closed-exc,hours-exc,merged,international,barrierefrei.1,action,image,urheberschaft,train,train_distance,short_distance,ignore
0,49,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,Bebelplatz,10117 Berlin,Mitte,WeihnachtsZauber Gendarmenmarkt GmbH\nTel.: 03...,25.11.24,23.12.24,"So-Do 12:00 - 22:00 Uhr\nFr, Sa 12:00 - 23:00 ...",mailto:info@weihnachtsmarkt-berlin.de,https://www.weihnachtsmarkt-berlin.de,Der WeihnachtsZauber Gendarmenmarkt an der Sta...,52.516664,13.394126,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,Bebelplatz,NaN,0,12:00-22:00,12:00-22:00,12:00-22:00,12:00-22:00,12:00-23:00,12:00-23:00,12:00-22:00,0,"24.12.23=12:00-18:00, 31.12.23=12:00-00:00",0,0,0,1,Weihnachsmarkt_Staatsoper.jpg,"Jörg Reichert, CC BY-SA 3.0",U Unter den Linden (U5),"2,099,298,949,036",1,0
1,53,Weihnachtsmarkt am Potsdamer Platz,Weihnachtsmarkt am Potsdamer Platz,Potsdamer Platz,10785 Berlin,Mitte,Bergmann Eventgastronomie,01.11.24,26.12.24,"10 bis 22 Uhr, 24. Dezember von 10 bis 16 Uhr",mailto:info@bergmannevent.de,http://www.bergmannevent.de,mit Rodelbahn und Hüttengaudi,52.509008,13.376262,Weihnachtsmarkt am Potsdamer Platz,Potsdamer Platz,NaN,1,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,0,24.12.23=10:00-16:00,0,0,0,0,Weihnachtsmarkt_PotsdamerPlatz.jpg,"KlemensM, CC0, via Wikimedia Commons","S+U Potsdamer Platz Bhf (U2, S1, S2, S25, S26)","434,891,072,201",1,0
2,54,Weihnachtsmarkt am Alexanderplatz,Weihnachtsmarkt am Alexanderplatz,Alexanderplatz,10178 Berlin,Mitte,Bergmann Eventgastronomie,25.11.24,26.12.24,"Täglich 10 bis 22 Uhr, 24. Dezember 10 bis 16 Uhr",mailto:info@bergmannevent.de,http://www.bergmannevent.de,"mit Almhütte und großer Weihnachtspyramide, \n...",52.521918,13.413215,Weihnachtsmarkt Alex,Alexanderplatz,NaN,1,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,0,24.12.23=10:00-16:00,0,0,0,1,Weihnachtsmarkt_Alex.jpg,"Pascal Volk from Berlin, Germany, CC BY-SA 2.0...",S+U Alexanderplatz Bhf,"999,260,150,342",1,0
3,55,Alt-Rixdorfer Weihnachtsmarkt,Alt-Rixdorfer Weihnachtsmarkt,Richardplatz,12055 Berlin,Neukölln,NaN,6.12.2024,8.12.2024,NaN,NaN,NaN,Leider liegen uns keine weiteren Informationen...,52.474222,13.445179,Alt-Rixdorfer Weihnachtsmarkt,Alt-Rixdorfer Weihnachtsmarkt,NaN,0,0,0,0,0,17:00-21:00,14:00-21:00,14:00-20:00,0,0,0,0,0,1,NaN,NaN,"U Karl-Marx-Str., S+U Neukölln",NaN,0,0


In [46]:
df.columns

Index(['id', 'name', 'shortname', 'strasse', 'plz_ort', 'bezirk',
       'veranstalter', 'von', 'bis', 'oeffnungszeiten', 'email', 'w3',
       'bemerkungen', 'lat', 'lng', 'rss_titel', 'rss_beschreibung',
       'barrierefrei', 'immer-kostenlos', 'Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa',
       'So', 'closed-exc', 'hours-exc', 'merged', 'international',
       'barrierefrei.1', 'action', 'image', 'urheberschaft', 'train',
       'train_distance', 'short_distance', 'ignore'],
      dtype='object')

In [47]:
#drop columns
col = ["merged", "veranstalter", "email", "rss_titel", "rss_beschreibung", "barrierefrei.1", "train_distance"]

df.drop(columns = col, inplace = True)

pd.set_option("display.max_columns", None)
df.head()


,id,name,shortname,strasse,plz_ort,bezirk,von,bis,oeffnungszeiten,w3,bemerkungen,lat,lng,barrierefrei,immer-kostenlos,Mo,Di,Mi,Do,Fr,Sa,So,closed-exc,hours-exc,international,action,image,urheberschaft,train,short_distance,ignore
0,49,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,Bebelplatz,10117 Berlin,Mitte,25.11.24,23.12.24,"So-Do 12:00 - 22:00 Uhr\nFr, Sa 12:00 - 23:00 ...",https://www.weihnachtsmarkt-berlin.de,Der WeihnachtsZauber Gendarmenmarkt an der Sta...,52.516664,13.394126,NaN,0,12:00-22:00,12:00-22:00,12:00-22:00,12:00-22:00,12:00-23:00,12:00-23:00,12:00-22:00,0,"24.12.23=12:00-18:00, 31.12.23=12:00-00:00",0,1,Weihnachsmarkt_Staatsoper.jpg,"Jörg Reichert, CC BY-SA 3.0",U Unter den Linden (U5),1,0
1,53,Weihnachtsmarkt am Potsdamer Platz,Weihnachtsmarkt am Potsdamer Platz,Potsdamer Platz,10785 Berlin,Mitte,01.11.24,26.12.24,"10 bis 22 Uhr, 24. Dezember von 10 bis 16 Uhr",http://www.bergmannevent.de,mit Rodelbahn und Hüttengaudi,52.509008,13.376262,NaN,1,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,0,24.12.23=10:00-16:00,0,0,Weihnachtsmarkt_PotsdamerPlatz.jpg,"KlemensM, CC0, via Wikimedia Commons","S+U Potsdamer Platz Bhf (U2, S1, S2, S25, S26)",1,0
2,54,Weihnachtsmarkt am Alexanderplatz,Weihnachtsmarkt am Alexanderplatz,Alexanderplatz,10178 Berlin,Mitte,25.11.24,26.12.24,"Täglich 10 bis 22 Uhr, 24. Dezember 10 bis 16 Uhr",http://www.bergmannevent.de,"mit Almhütte und großer Weihnachtspyramide, \n...",52.521918,13.413215,NaN,1,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,0,24.12.23=10:00-16:00,0,1,Weihnachtsmarkt_Alex.jpg,"Pascal Volk from Berlin, Germany, CC BY-SA 2.0...",S+U Alexanderplatz Bhf,1,0
3,55,Alt-Rixdorfer Weihnachtsmarkt,Alt-Rixdorfer Weihnachtsmarkt,Richardplatz,12055 Berlin,Neukölln,6.12.2024,8.12.2024,NaN,NaN,Leider liegen uns keine weiteren Informationen...,52.474222,13.445179,NaN,0,0,0,0,0,17:00-21:00,14:00-21:00,14:00-20:00,0,0,0,1,NaN,NaN,"U Karl-Marx-Str., S+U Neukölln",0,0


In [48]:
#add column bemerkungen_en
df["bemerkungen_en"] = None

#new order of data frame columns
df = df[["id","name", "shortname", "strasse", "bezirk", "plz_ort", "von", "bis", "oeffnungszeiten", "Mo", "Di", "Mi", "Do", "Fr", "Sa", "So", "closed-exc", "hours-exc", "w3", "bemerkungen", "bemerkungen_en", "lat", "lng", "ignore", "immer-kostenlos", "international", "barrierefrei", "action", "short_distance", "train", "image", "urheberschaft"]]

df.head()

,id,name,shortname,strasse,bezirk,plz_ort,von,bis,oeffnungszeiten,Mo,Di,Mi,Do,Fr,Sa,So,closed-exc,hours-exc,w3,bemerkungen,bemerkungen_en,lat,lng,ignore,immer-kostenlos,international,barrierefrei,action,short_distance,train,image,urheberschaft
0,49,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,Bebelplatz,Mitte,10117 Berlin,25.11.24,23.12.24,"So-Do 12:00 - 22:00 Uhr\nFr, Sa 12:00 - 23:00 ...",12:00-22:00,12:00-22:00,12:00-22:00,12:00-22:00,12:00-23:00,12:00-23:00,12:00-22:00,0,"24.12.23=12:00-18:00, 31.12.23=12:00-00:00",https://www.weihnachtsmarkt-berlin.de,Der WeihnachtsZauber Gendarmenmarkt an der Sta...,None,52.516664,13.394126,0,0,0,NaN,1,1,U Unter den Linden (U5),Weihnachsmarkt_Staatsoper.jpg,"Jörg Reichert, CC BY-SA 3.0"
1,53,Weihnachtsmarkt am Potsdamer Platz,Weihnachtsmarkt am Potsdamer Platz,Potsdamer Platz,Mitte,10785 Berlin,01.11.24,26.12.24,"10 bis 22 Uhr, 24. Dezember von 10 bis 16 Uhr",10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,0,24.12.23=10:00-16:00,http://www.bergmannevent.de,mit Rodelbahn und Hüttengaudi,None,52.509008,13.376262,0,1,0,NaN,0,1,"S+U Potsdamer Platz Bhf (U2, S1, S2, S25, S26)",Weihnachtsmarkt_PotsdamerPlatz.jpg,"KlemensM, CC0, via Wikimedia Commons"
2,54,Weihnachtsmarkt am Alexanderplatz,Weihnachtsmarkt am Alexanderplatz,Alexanderplatz,Mitte,10178 Berlin,25.11.24,26.12.24,"Täglich 10 bis 22 Uhr, 24. Dezember 10 bis 16 Uhr",10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,10:00-22:00,0,24.12.23=10:00-16:00,http://www.bergmannevent.de,"mit Almhütte und großer Weihnachtspyramide, \n...",None,52.521918,13.413215,0,1,0,NaN,1,1,S+U Alexanderplatz Bhf,Weihnachtsmarkt_Alex.jpg,"Pascal Volk from Berlin, Germany, CC BY-SA 2.0..."
3,55,Alt-Rixdorfer Weihnachtsmarkt,Alt-Rixdorfer Weihnachtsmarkt,Richardplatz,Neukölln,12055 Berlin,6.12.2024,8.12.2024,NaN,0,0,0,0,17:00-21:00,14:00-21:00,14:00-20:00,0,0,NaN,Leider liegen uns keine weiteren Informationen...,None,52.474222,13.445179,0,0,0,NaN,1,0,"U Karl-Marx-Str., S+U Neukölln",NaN,NaN


In [49]:
#adjust time format in columns Mo - So
weekdays = ["Mo", "Di", "Mi", "Do", "Fr", "Sa", "So"]

#split columns
for col in weekdays:
    df[col] = df[col].str.split("-")
    #add space before and after hyphen
    df[col] = df[col].str[0] + " - " + df[col].str[1]

df.head()

,id,name,shortname,strasse,bezirk,plz_ort,von,bis,oeffnungszeiten,Mo,Di,Mi,Do,Fr,Sa,So,closed-exc,hours-exc,w3,bemerkungen,bemerkungen_en,lat,lng,ignore,immer-kostenlos,international,barrierefrei,action,short_distance,train,image,urheberschaft
0,49,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,WeihnachtsZauber Gendarmenmarkt an der Staatsoper,Bebelplatz,Mitte,10117 Berlin,25.11.24,23.12.24,"So-Do 12:00 - 22:00 Uhr\nFr, Sa 12:00 - 23:00 ...",12:00 - 22:00,12:00 - 22:00,12:00 - 22:00,12:00 - 22:00,12:00 - 23:00,12:00 - 23:00,12:00 - 22:00,0,"24.12.23=12:00-18:00, 31.12.23=12:00-00:00",https://www.weihnachtsmarkt-berlin.de,Der WeihnachtsZauber Gendarmenmarkt an der Sta...,None,52.516664,13.394126,0,0,0,NaN,1,1,U Unter den Linden (U5),Weihnachsmarkt_Staatsoper.jpg,"Jörg Reichert, CC BY-SA 3.0"
1,53,Weihnachtsmarkt am Potsdamer Platz,Weihnachtsmarkt am Potsdamer Platz,Potsdamer Platz,Mitte,10785 Berlin,01.11.24,26.12.24,"10 bis 22 Uhr, 24. Dezember von 10 bis 16 Uhr",10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,0,24.12.23=10:00-16:00,http://www.bergmannevent.de,mit Rodelbahn und Hüttengaudi,None,52.509008,13.376262,0,1,0,NaN,0,1,"S+U Potsdamer Platz Bhf (U2, S1, S2, S25, S26)",Weihnachtsmarkt_PotsdamerPlatz.jpg,"KlemensM, CC0, via Wikimedia Commons"
2,54,Weihnachtsmarkt am Alexanderplatz,Weihnachtsmarkt am Alexanderplatz,Alexanderplatz,Mitte,10178 Berlin,25.11.24,26.12.24,"Täglich 10 bis 22 Uhr, 24. Dezember 10 bis 16 Uhr",10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,10:00 - 22:00,0,24.12.23=10:00-16:00,http://www.bergmannevent.de,"mit Almhütte und großer Weihnachtspyramide, \n...",None,52.521918,13.413215,0,1,0,NaN,1,1,S+U Alexanderplatz Bhf,Weihnachtsmarkt_Alex.jpg,"Pascal Volk from Berlin, Germany, CC BY-SA 2.0..."
3,55,Alt-Rixdorfer Weihnachtsmarkt,Alt-Rixdorfer Weihnachtsmarkt,Richardplatz,Neukölln,12055 Berlin,6.12.2024,8.12.2024,NaN,NaN,NaN,NaN,NaN,17:00 - 21:00,14:00 - 21:00,14:00 - 20:00,0,0,NaN,Leider liegen uns keine weiteren Informationen...,None,52.474222,13.445179,0,0,0,NaN,1,0,"U Karl-Marx-Str., S+U Neukölln",NaN,NaN


In [ ]:
#csv
df.to_csv("markets_zusatz_proc.csv", index = False)